# Invoke Bedrock with HealthScribe Summary

> *This notebook is compatible with SageMaker kernels `Data Science 3.0` or `conda_python3` on an `ml.t3.medium` instance*

## Setup
Before running the remaining cells of this notebook, we need to confirm you have a version of boto3 that supports Bedrock. Any version newer than **`1.28.58`** should be fine. Run the following cell to check the installed version. 

In [ ]:
!pip list | grep boto3

(Optional) Only run this cell if you have an older version of boto3 that does not support Bedrock.

In [ ]:
!pip install botocore boto3 --upgrade

#### List available models in Bedrock

This cell calls **`list_foundation_models`** API in Bedrock filtering only models provided by **`Anthropic`**.

In [ ]:
import boto3, json

bedrock = boto3.client('bedrock', 'us-west-2')
response = bedrock.list_foundation_models(byProvider='anthropic')
print(json.dumps(response.get('modelSummaries'), indent=2))

---

## 3. Combining HealthScribe with Bedrock

In this step we are going to invoke model **`Claude v3 Sonnet`** from **`Anthropic`** on Amazon Bedrock asking the model to provide ICD-10 codes and procedure codes based on the plan generated by AWS HealthScribe. This could be an alternative solution to using Comprehend Medical, but also complements what Comprehend provides by enabling you to extract CPT codes that are currently not supported by the service.

In [ ]:
bedrock_runtime = boto3.client('bedrock-runtime', 'us-west-2')

# retrieve plan from previous notebook
%store -r plan
print(plan)

This helper function encapsulates Bedrock **`invoke_model`** API call:

In [ ]:
def generate_message(prompt, max_tokens, temperature):
    
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    user_message =  {"role": "user", "content": prompt}
    messages = [user_message]
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temperature
        }
    )
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    return response_body

Let's first invoke **`Claude v3 Sonnet`** with a simple test prompt:

In [ ]:
prompt = "what is the CPT code for an x-ray of their lower back?"

response_body = generate_message(prompt, 1000, 0)

print(response_body.get('content')[0].get('text'))

### 3.1. Send HealthScribe Summary Plan to Bedrock to Extract ICD-10 Codes

In this step we are going to invoke model **`Claude v3 Sonnet`** to provide ICD-10 codes for diagnostic components mentioned in the plan generated by HealthScribe like we did in the previous notebook.

In [ ]:
prompt = f"""

For each one of the bullet points defined in <plan></plan> XML tags between triple backslashes, 
list the most common ICD-10 codes applicable. If there are no codes for a recommendation, do not include them in your response.
Present your output using the same JSON format used by Amazon Comprehend Medical.

<plan>{plan}</plan>

"""

response_body = generate_message(prompt, 1000, 0)

print(response_body.get('content')[0].get('text'))

### 3.2. Send HealthScribe Summary Plan to Bedrock to Extract CPT Codes

No we are going to invoke model **`Claude v3 Sonnet` to provide procedure codes for all relevant procedures listed in the plan generated by HealthScribe. This is something currently not supported by Comprehend Medical.

In [ ]:
prompt = f"""

For each one of the bullet points defined in <plan></plan> XML tags between triple backslashes, 
list the most common CPT codes applicable. If there are no codes for a recommendation, do not include them in your response.

<plan>{plan}</plan>

"""

response_body = generate_message(prompt, 1000, 0)

print(response_body.get('content')[0].get('text'))